# Test NSEG dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Masking, Bidirectional
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("/content/drive/MyDrive/FVAB/dataset_de_LDS_SEEDIV.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [8]:
df[df.isnull()==True] = -10

In [9]:
def preprocessing_for_LSTM(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = to_categorical(y_train)
    X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Subject dependent

In [10]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [11]:
df

0          1          2          3          4  \
id_user session video                                                          
10      1       1      25.622467  22.612988  21.225053  19.946429  19.266408   
                2      25.496412  22.407150  20.974761  20.547296  20.101432   
                3      25.302885  22.570660  21.633718  21.230311  21.011355   
                4      25.939116  22.808869  21.332588  19.427715  18.515616   
                5      25.503216  22.350826  21.157540  19.464702  18.714055   
...                          ...        ...        ...        ...        ...   
9       3       20     25.031688  22.242642  21.493354  20.546762  20.240579   
                21     25.676278  22.638414  21.002611  20.324261  20.132503   
                22     25.323145  22.412124  21.821257  20.533943  20.393856   
                23     24.879851  22.047250  21.202716  20.121354  19.557465   
                24     25.481525  22.404174  21.688045  20.494014  20.080786   

                               5          6          7          8          9  \
id_user session video                                                          
10      1       1      23.396372  20.964809  20.035117  17.815367  16.887869   
                2      23.023210  21.083666  19.698849  17.761071  16.874142   
                3      23.156935  21.018987  20.335182  18.201790  17.557549   
                4      22.954357  20.906463  20.060412  17.828035  16.748080   
                5      23.024172  20.880395  20.269262  17.854670  16.799909   
...                          ...        ...        ...        ...        ...   
9       3       20     21.793385  18.860982  18.035320  16.896775  16.196043   
                21     21.736976  18.712277  17.463331  16.739365  16.433142   
                22     21.565260  18.816661  18.326217  17.525892  17.510356   
                23     21.748668  18.707098  17.733498  16.605061  15.808999   
                24     21.794618  18.929078  18.257980  16.976043  16.482274   

                       ...  1590  1591  1592  1593  1594  1595  1596  1597  \
id_user session video  ...                                                   
10      1       1      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                2      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                3      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                4      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                5      ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
...                    ...   ...   ...   ...   ...   ...   ...   ...   ...   
9       3       20     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                21     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                22     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                23     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   
                24     ... -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0 -10.0   

                       1598  1599  
id_user session video              
10      1       1     -10.0 -10.0  
                2     -10.0 -10.0  
                3     -10.0 -10.0  
                4     -10.0 -10.0  
                5     -10.0 -10.0  
...                     ...   ...  
9       3       20    -10.0 -10.0  
                21    -10.0 -10.0  
                22    -10.0 -10.0  
                23    -10.0 -10.0  
                24    -10.0 -10.0  

[1080 rows x 1601 columns]

In [24]:
accuracy_test = []
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    del model
    model = Sequential()
    model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
    model.add(Bidirectional(GRU(units=32, input_shape=(X_trainSD.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=300, batch_size=4)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

Epoch 1/300
120/120 [==============================] - 14s 65ms/step - loss: 1.4340 - accuracy: 0.3042
Epoch 2/300
120/120 [==============================] - 7s 55ms/step - loss: 1.3607 - accuracy: 0.3208
Epoch 3/300
120/120 [==============================] - 8s 64ms/step - loss: 1.3781 - accuracy: 0.3042
Epoch 4/300
120/120 [==============================] - 7s 56ms/step - loss: 1.3610 - accuracy: 0.3250
Epoch 5/300
120/120 [==============================] - 8s 63ms/step - loss: 1.3674 - accuracy: 0.3167
Epoch 6/300
120/120 [==============================] - 8s 71ms/step - loss: 1.3568 - accuracy: 0.3333
Epoch 7/300
120/120 [==============================] - 8s 65ms/step - loss: 1.3552 - accuracy: 0.3208
Epoch 8/300
120/120 [==============================] - 8s 64ms/step - loss: 1.3466 - accuracy: 0.3208
Epoch 9/300
120/120 [==============================] - 7s 55ms/step - loss: 1.3543 - accuracy: 0.3000
Epoch 10/300
120/120 [==============================] - 8s 65ms/step - loss: 1.35

KeyboardInterrupt: ignored

In [ ]:
print("ACCURACY: ", np.array(accuracy_test).mean())

# Subject-Biased Experiment


In [31]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=104)
X_train, X_test, y_train, y_test = preprocessing_for_LSTM(X_train, X_test, y_train, y_test)

In [33]:
del model
model = Sequential()
model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
model.add(Bidirectional(GRU(units=128, input_shape=(X_trainSD.shape[1], 1))))
model.add(Dense(units=4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=300, batch_size=16)

Epoch 1/100
54/54 [==============================] - 12s 95ms/step - loss: 1.4025 - accuracy: 0.2975
Epoch 2/100
54/54 [==============================] - 4s 83ms/step - loss: 1.3877 - accuracy: 0.3218
Epoch 3/100
54/54 [==============================] - 5s 97ms/step - loss: 1.3795 - accuracy: 0.2894
Epoch 4/100
54/54 [==============================] - 4s 83ms/step - loss: 1.3855 - accuracy: 0.3021
Epoch 5/100
54/54 [==============================] - 5s 83ms/step - loss: 1.3796 - accuracy: 0.3356
Epoch 6/100
54/54 [==============================] - 5s 96ms/step - loss: 1.3633 - accuracy: 0.3345
Epoch 7/100
54/54 [==============================] - 4s 83ms/step - loss: 1.3634 - accuracy: 0.3368
Epoch 8/100
54/54 [==============================] - 5s 99ms/step - loss: 1.3689 - accuracy: 0.3194
Epoch 9/100
54/54 [==============================] - 5s 89ms/step - loss: 1.3711 - accuracy: 0.3287
Epoch 10/100
54/54 [==============================] - 4s 82ms/step - loss: 1.3814 - accuracy: 0.322

In [34]:
loss, accuracy = model.evaluate(X_test, y_test)

7/7 [==============================] - 3s 52ms/step - loss: 1.1925 - accuracy: 0.4583


# Subject Indipendent

In [ ]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

# TO DO: da implementare kfold
https://stackoverflow.com/questions/48085182/cross-validation-in-keras


"""
from sklearn.model_selection import RepeatedKFold, cross_val_score
from tensorflow.keras.models import * 
from tensorflow.keras.layers import * 
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

def buildmodel():
    model= Sequential([
        Dense(10, activation="relu"),
        Dense(5, activation="relu"),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    return(model)

estimator= KerasRegressor(build_fn=buildmodel, epochs=100, batch_size=10, verbose=0)
kfold= RepeatedKFold(n_splits=5, n_repeats=100)
results= cross_val_score(estimator, x, y, cv=kfold, n_jobs=2)  # 2 cpus
results.mean()  # Mean MSE
"""

In [ ]:
scores_test = []
logo = LeaveOneGroupOut()
for i in range(1,4):
    X_SI,y_SI,groups = subject_independent_split(df,i)
    X_SI = np.array(X_SI).reshape((X_SI.shape[0], X_SI.shape[1], 1))
    y_SI = to_categorical(y_SI)
    del model
    model = Sequential()
    model.add(LSTM(units=64, input_shape=(int(X_SI.shape[1])-15, 1)))
    model.add(Dense(units=4, activation='relu'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    scores_gbc = cross_val_score(model, X_SI, y_SI, cv=logo, verbose=1 , groups = groups, n_jobs = -1)
    scores_test.append(rfSD.scores.mean())

In [ ]:
print("TEST:", np.array(scores_test).mean())